In [1]:
import sys
from time import sleep
from time import gmtime, strftime
import mmap
import resource
import numpy as np
import os
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import FileLink, FileLinks
from IPython.display import clear_output
import datetime
%matplotlib inline

# This sets up the FPGA and maps some memory locations to variables we can access
# Don't change anything here unless you enjoy things not working.
os.system('cat /root/system_wrapper.bit > /dev/xdevcfg')
#os.system('cat /opt/redpitaya/fpga/pet_rebuild.bit > /dev/xdevcfg')

fd = os.open('/dev/mem', os.O_RDWR)
mm = []
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41210000)) #0
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41300000)) #1
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41200000)) #2
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41230000)) #3
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41240000)) #4
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41250000)) #5
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41220000)) #6
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41260000)) #7
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41270000)) #8
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41280000)) #9
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41290000)) #10
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x412A0000)) #11
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x412B0000)) #12
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x412C0000)) #13
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x412D0000)) #14
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x42220000)) #15
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x412E0000)) #16
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x412F0000)) #17
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41310000)) #18
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41320000)) #19
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41330000)) #20
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41340000)) #21

lat_in  = np.frombuffer(mm[0], dtype='uint32') 
lat_out = np.frombuffer(mm[1], dtype='uint32') 
rot_in  = np.frombuffer(mm[2], dtype='uint32') 
ch1_threshold = np.frombuffer(mm[3], dtype='uint32') 
ch2_threshold = np.frombuffer(mm[4], dtype='uint32') 
coincidence_settings = np.frombuffer(mm[5], dtype='uint16') 
threshold_invert_settings = np.frombuffer(mm[6], dtype='uint32') 
coincidence_timeout_counts  = np.frombuffer(mm[7], dtype='uint32') 
counts  =  np.frombuffer(mm[8], dtype='uint32') 
uthreshold_counts =  np.frombuffer(mm[9], dtype='uint32') 
coincidence_status =  np.frombuffer(mm[10], dtype='uint32') 
raw_input = np.frombuffer(mm[11],count=2,dtype='int16') # ADC data is stored as a signed integer
#sensor_readout = np.frombuffer(mm[11],dtype='uint32')
sensor_readout = np.frombuffer(mm[11],offset=8,count=1,dtype='uint32') # This skips the first 4 bytes, getting to the readout info.
threshold_detection = np.frombuffer(mm[12],dtype='uint32')
#adc_module_signal = np.frombuffer(mm[13],dtype='uint32')
offset_settings_ch2 = np.frombuffer(mm[13],dtype='uint32')
direct_adc = np.frombuffer(mm[14],dtype='uint32')
offset_settings = np.frombuffer(mm[15],dtype='uint32')
dac_settings = np.frombuffer(mm[16],dtype='uint32')
rotation_settings = np.frombuffer(mm[17],dtype='int32')

PHA_readout = np.frombuffer(mm[18],dtype='uint32')
PHA_stored_pulses = np.frombuffer(mm[19],dtype='uint32')
toggler  = np.frombuffer(mm[20],dtype='uint32')
scan_runtime = np.frombuffer(mm[21],dtype='uint32')


In [2]:
ch1_threshold[0] = 8500
ch1_threshold[2] = 9000

ch2_threshold[0] = 8400
ch2_threshold[2] = 9000

offset_settings[0] = 8500
offset_settings[2] = 8400

threshold_invert_settings[0] = 1
threshold_invert_settings[2] = 1

In [3]:
import math

def seconds_to_ticks(ts):
    return math.ceil(ts*1.25e8) #The Red Pitaya has a 125 MHz clock

def get_counts(time = 1,show_counts=None):
    global coincidence_settings
    coincidence_settings[0] = 1
    sleep(0.001)
    global scan_runtime
    scan_runtime[0] = seconds_to_ticks(time)
    coincidence_settings[0] = 3
    sleep(0.001)
    coincidence_settings[0] = 2
    for i in range(time):
        sleep(1)
        with show_counts:
            clear_output(wait=True)
            print("Coincidences: {}\nCh1 Counts: {}\nCh2 Counts: {}".format(coincidence_timeout_counts[0],counts[0],counts[2]))
    coinc = coincidence_timeout_counts[0]
    ch1 = counts[0]
    ch2 = counts[2]
    coincidence_settings[0] = 3
    sleep(0.001)
    coincidence_settings[0] = 0
    return coinc,ch1,ch2

In [4]:
import threading
import time
class output_readout():
    def __init__(self,*args,**kwargs):
        print(args,kwargs)
        self.name = args[0]
        self.variable = args[1]
        self.widget = widgets.Label(value=self.name +": "+ str(eval(self.variable)))
        self.running = True
        self.update_rate = .1
        self.thread = threading.Thread(target=self.update_thread)
        self.thread.start()
        for key,val in kwargs.items():
            setattr(self,key,val)
        #if args[2] is not None:
        #    self.type = args[2]
    def __exit__(self):
        self.thread.stop()
    def get(self):
        return eval(self.variable)
    def update(self):
       # if self.type == "standard":
        if "single_bit" in vars(self):
            self.widget.value = self.name +": "+ str(  eval(self.variable) & 2**self.single_bit != 0)
        elif "dictionary" in vars(self):
            if eval(self.variable) in self.dictionary:
                self.widget.value = self.name +": "+ str(  self.dictionary[eval(self.variable)])
            else:
                self.widget.value = "Unknown state: " + str(eval(self.variable))
        elif "conversion" in vars(self):
            self.widget.value = self.name +": {:.3f} V ".format(eval(self.variable)/self.conversion)
        else:
            self.widget.value = self.name +": "+ str(eval(self.variable))
       # elif self.type == "ADC":
       #     self.widget.value = self.name +": "+ str(eval(self.variable)/2**10)
    def update_thread(self):
        while True:
            if self.running:
                self.update()
            time.sleep(self.update_rate)
    def stop(self):
        self.running=False
    def start(self):
        self.running=True

In [5]:
class continuous_scan():
    def __init__(self,*args,**kwargs):
        print(args,kwargs)
        self.name = "scan"
        self.widget = widgets.Label(value=self.name)
        self.running = True
        self.update_rate = 1
        self.thread = threading.Thread(target=self.update_thread)
        self.thread.start()
        for key,val in kwargs.items():
            setattr(self,key,val)
        #if args[2] is not None:
        #    self.type = args[2]
    def __exit__(self):
        self.thread.stop()
    def get(self):
        return eval(self.variable)
    def update(self):
        global coincidence_settings
        global scan_runtime
        global coincidence_timeout_counts
        global counts
        global uthreshold_counts
        self.widget.value = "Coincidences: {}\nCh1 Counts: {}\nCh2 Counts: {}".format(coincidence_timeout_counts[0],counts[0]-uthreshold_counts[0],counts[2]-uthreshold_counts[2])
        coincidence_settings[0] = 1
        sleep(0.001)
        scan_runtime[0] = seconds_to_ticks(1)
        coincidence_settings[0] = 3
        sleep(0.001)
        coincidence_settings[0] = 2
        
    def update_thread(self):
        while True:
            if self.running:
                self.update()
            time.sleep(self.update_rate)
    def stop(self):
        self.running=False
    def start(self):
        self.running=True

In [6]:
class input_readout():
    def __init__(self,*args,**kwargs):
        print(args,kwargs)
        self.name = args[0]
        self.variable = args[1]
        for key,val in kwargs.items():
            setattr(self,key,val)
        self.widget = widgets.Label(value=self.name +": "+ str(eval(self.variable)))
        if "single_bit" in vars(self):
            if "invert" in vars(self):
                self.set_widget = widgets.Checkbox(value=self.invert,description = "Set " + (self.name), indent=False)
            else:
                self.set_widget = widgets.Checkbox(value=False,description = "Set " + (self.name), indent=False)
        elif "dictionary" in vars(self):
            self.set_widget = widgets.Dropdown(options = list(zip(list(self.dictionary.values()),list(self.dictionary))),description = "Set " + (self.name))
        else:
            self.set_widget = widgets.IntText(description = "Set",value = eval(self.variable),step=10)
        self.update_value=True
        self.running = True
        if "update_rate" not in vars(self):
            self.update_rate = .5
        self.thread = threading.Thread(target=self.update_thread)
        self.thread.start()

        #if args[2] is not None:
        #    self.type = args[2]
    def __exit__(self):
        self.thread.stop()
    def get(self):
        return eval(self.variable)
    def update(self):
        if(self.update_value):
            if "single_bit" in vars(self):
                if self.set_widget.value:
                    self.widget.value = str(eval(self.variable) | (1<<self.single_bit))
                    exec(self.variable +"="+ str(eval(self.variable) | (1<<self.single_bit)))
                else:
                    self.widget.value = str(eval(self.variable) & ~(1<<self.single_bit))
                    exec(self.variable +"="+ str(eval(self.variable) & ~(1<<self.single_bit)))
            elif "conversion" in vars(self):
                self.widget.value = self.name +": {:.3f} V ".format(0.5 - eval(self.variable)/self.conversion)
                exec(self.variable +"="+ str(self.set_widget.value))
            elif "dictionary" in vars(self):
                 self.widget.value = self.name +": "+ str(eval(self.variable))
                 exec(self.variable +"="+ str(self.set_widget.value))
            else:
                self.widget.value = self.name +": "+ str(eval(self.variable))
                exec(self.variable +"="+ str(self.set_widget.value))
    def update_thread(self):
        while True:
            if self.running:
                self.update()
            time.sleep(self.update_rate)
    def stop(self):
        self.running=False
    def start(self):
        self.running=True

In [7]:
class divider():
    def __init__(self):
        self.widget= widgets.HTML("<hr>")

In [8]:
coincidence_dict = {0:"Idle",
                    1:"Signal on ch 1",
                    2:"Signal on ch 2",
                    3:"Coincidence detected",
                    4:"Timeout no signal on 2",
                    5:"Coincidence detection loop finished",
                    6:"Error",
                    7:"Ch1 over threshold",
                    8:"Ch2 over threshold",
                    9:"Timeout no signal on 1",
                    10:"Recover"}
offset_settings[0] = 0
offset_settings[2] = 0
rotation_dict =    {0:"Idle",
                    1:"Running",
                    2:"Run Finished",
                    3:"Resetting",
                    4:"Timeout before reset finished",
                    19:"Resetting: First pass detected",
                    35:"Resetting: Second pass",
                    36:"Reset Timeout"}
lateral_dict =     {0:"Idle",
                    1:"Running",
                    2:"Run Finished",
                    3:"Resetting",
                    4:"Error",
                    19:"Resetting: First pass detected",
                    35:"Resetting: Second pass",
                    36:"Reset Timeout"}
DAC_dict =          {0:"Ch 1 Low Threshold",
                     1:"Ch 1 High Threshold",
                     2:"Ch 1 Input",
                     3:"Ch 2 Input", #4 DAC lateral signals
                     6:"Ch 2 Low Threshold", # 5 DAC rot signals
                     7:"Ch 2 High Threshold", #8 Lower threshold met extended
                     11:"Ch 1 Filtered", #9 Upper threshold met extended
                     10:"Ch 2 Filtered", #9 Upper threshold met extended
                     128: "Ch 1 Both Thresholds",
                     129: "Ch 2 Both Thresholds"}

In [9]:
def scan():
    while True:
        t=1
        coincidence_settings[0] = 1
        sleep(0.001)
        scan_runtime[0] = seconds_to_ticks(t)
        coincidence_settings[0] = 3
        sleep(0.001)
        coincidence_settings[0] = 2
        for i in range(t):
            sleep(1)
            with general_output:
                clear_output(wait=True)
                print("Coincidences: {}\nCh1 Counts: {}\nCh2 Counts: {}".format(coincidence_timeout_counts[0],counts[0]-uthreshold_counts[0],counts[2]-uthreshold_counts[2]))
        coinc = coincidence_timeout_counts[0]
        ch1 = counts[0]
        ch2 = counts[2]
        coincidence_settings[0] = 3
        sleep(0.001)
        coincidence_settings[0] = 0

In [10]:
def lat_reset():
    lat_in[0] = lat_in[0]|0b100
    sleep(0.001)
    lat_in[0] &= ~0b001 # Guarantees set to zero
    while(lat_out[0] ==3):
        pass # waits while the setup is moving
    sleep(0.001)
    if(lat_out[0] == 2):
        lat_in[0] = 8
        sleep(0.01)
        lat_in[0] = 0  
    if(lat_out[0] == 4):
        raise RuntimeError("Controller returned error code")
    sleep(1.1)
    
def lat_move(dr,dist):
   # print("initial:",lat_in[0])
    try:
        lat_in[2] = dist
    except e:
        print("Invalid distance of ",dist)
        raise e
    if dr == 0:
        lat_in[0] = lat_in[0]|0b10    
    else:
        lat_in[0] &= ~0b10
    #print("After dr set",lat_in[0])
    lat_in[0] = lat_in[0]|0b1
    #print("After start set",lat_in[0])
    sleep(0.001)
    lat_in[0] &= ~0b1 # Guarantees set to zero
    #print("After toggle", lat_in[0])
    while(lat_out[0] ==1):
        pass # waits while the setup is moving
    #print(lat_in[0])
    if(lat_out[0] == 2):
        lat_in[0] = 8
        sleep(0.001)
        lat_in[0] = 0
    #print("End",lat_in[0])
    if(lat_out[0] == 4):
        raise RuntimeError("Controller returned error code")
        
def rot_reset():
    rot_in[0] = rot_in[0]|0b100
    sleep(0.001)
    rot_in[0] &= ~0b001 # Guarantees set to zero
    while(rotation_settings[0] >=3 or rotation_settings[0] ==19 ):
        pass # waits while the setup is moving
    if(rotation_settings[0] == 2):
        rot_in[0] = 8
        sleep(0.001)
        rot_in[0] = 0  
    sleep(1.1)

In [11]:
lat_reset()
#rot_reset()

if "fpga_readouts" in locals(): # closes the existing widget if it exists
    try:
        for item in children:
            item.close()
        fpga_readouts.clear()
    except:
        pass
fpga_readouts = []

hrule = divider()

fpga_readouts.append(output_readout("Ch1 Counts","counts[0]"))
fpga_readouts.append(input_readout("Ch1 Lower Threshold","ch1_threshold[0]",conversion = 2**14))
fpga_readouts.append(input_readout("Ch1 Upper Threshold","ch1_threshold[2]",conversion = 2**14))
fpga_readouts.append(input_readout("Ch1 offset","offset_settings[0]",conversion = 2**14))
fpga_readouts.append(output_readout("Ch1 Reading","direct_adc[0]"))
fpga_readouts.append(hrule)
fpga_readouts.append(output_readout("Ch2 Counts","counts[2]"))
fpga_readouts.append(input_readout("Ch2 Lower Threshold","ch2_threshold[0]",conversion = 2**14))
fpga_readouts.append(input_readout("Ch2 Upper Threhold", "ch2_threshold[2]",conversion = 2**14))
fpga_readouts.append(input_readout("Ch2 offset","offset_settings_ch2[0]",conversion = 2**14))
fpga_readouts.append(output_readout("Ch2 Reading","direct_adc[2]"))
fpga_readouts.append(hrule)
fpga_readouts.append(output_readout("Coincidences","coincidence_timeout_counts[0]"))
fpga_readouts.append(output_readout("Timeouts","coincidence_timeout_counts[2]"))
fpga_readouts.append(output_readout("Ch1 Overthreshold Counts","uthreshold_counts[0]"))
fpga_readouts.append(output_readout("Ch2 Overthreshold Counts","uthreshold_counts[2]"))
fpga_readouts.append(hrule)
i1 = input_readout("Out 1 Select","dac_settings[2]",dictionary = DAC_dict)
i1.set_widget.value = 1
i2 = input_readout("Out 2 Select","dac_settings[0]",dictionary = DAC_dict)
i2.set_widget.value = 2
fpga_readouts.append(i1)
fpga_readouts.append(i2)
fpga_readouts.append(continuous_scan())


children = []
for item in fpga_readouts:
    if "set_widget" in vars(item):
        children.append(widgets.HBox((item.widget,item.set_widget)))           
    else:
        children.append(item.widget)
data_pane = widgets.VBox(children)
display(data_pane)

general_output = widgets.Output()
display(general_output)
lat_move(1,0) #something gets stuck and eats the first move command right now.  Argh.
lat_move(1,40000)
#scan()

('Ch1 Counts', 'counts[0]') {}
('Ch1 Lower Threshold', 'ch1_threshold[0]') {'conversion': 16384}
('Ch1 Upper Threshold', 'ch1_threshold[2]') {'conversion': 16384}
('Ch1 offset', 'offset_settings[0]') {'conversion': 16384}
('Ch1 Reading', 'direct_adc[0]') {}
('Ch2 Counts', 'counts[2]') {}
('Ch2 Lower Threshold', 'ch2_threshold[0]') {'conversion': 16384}
('Ch2 Upper Threhold', 'ch2_threshold[2]') {'conversion': 16384}
('Ch2 offset', 'offset_settings_ch2[0]') {'conversion': 16384}
('Ch2 Reading', 'direct_adc[2]') {}
('Coincidences', 'coincidence_timeout_counts[0]') {}
('Timeouts', 'coincidence_timeout_counts[2]') {}
('Ch1 Overthreshold Counts', 'uthreshold_counts[0]') {}
('Ch2 Overthreshold Counts', 'uthreshold_counts[2]') {}
('Out 1 Select', 'dac_settings[2]') {'dictionary': {0: 'Ch 1 Low Threshold', 1: 'Ch 1 High Threshold', 2: 'Ch 1 Input', 3: 'Ch 2 Input', 6: 'Ch 2 Low Threshold', 7: 'Ch 2 High Threshold', 129: 'Ch 2 Both Thresholds', 10: 'Ch 2 Filtered', 11: 'Ch 1 Filtered', 128: 'Ch

Output()

----